#1.

Dorim sa aratam ca daca A,B si C sunt variabile aleatoare discrete, atunci $A \perp C \mid B$ are loc daca si numai daca exista doua functii pozitive $\phi_{A,B}$, $\phi_{B,C}$ si o constanta de normalizare $k > 0$ astfel incat:

$$
P(A = x_A, B = x_B, C = x_C) = \frac{1}{k} \phi_{A,B}(x_A, x_B) \phi_{B,C}(x_B, x_C), \quad \forall x_A, x_B, x_C.
$$

---

#### Pasul 1: Independenta conditionata $A \perp C \mid B$

Independenta conditionata $A \perp C \mid B$ implica faptul ca distributia comuna $P(A, C \mid B)$ poate fi descompusa in produsul probabilitatilor marginale conditionate:

$$
P(A = x_A, C = x_C \mid B = x_B) = P(A = x_A \mid B = x_B) P(C = x_C \mid B = x_B), \quad \forall x_A, x_B, x_C.
$$

Din definitia probabilitatii conditionate, avem:

$$
P(A = x_A, C = x_C \mid B = x_B) = \frac{P(A = x_A, B = x_B, C = x_C)}{P(B = x_B)}.
$$

---

#### Pasul 2: Formularea cu functii pozitive si normalizare

Pp ca exista functiile pozitive $\phi_{A,B}$ si $\phi_{B,C}$ astfel incat:

$$
P(A = x_A, B = x_B, C = x_C) = \frac{1}{k} \phi_{A,B}(x_A, x_B) \phi_{B,C}(x_B, x_C), \quad \forall x_A, x_B, x_C.
$$

Pentru a verifica aceasta forma, trebuie sa demonstram ca se respecta urmatoarele proprietati:

1. **Distributia e pozitiva**:
   $$P(A = x_A, B = x_B, C = x_C) > 0, \quad \forall x_A, x_B, x_C.$$

2. **Normalizarea**:
   $$\sum_{x_A, x_B, x_C} P(A = x_A, B = x_B, C = x_C) = 1.$$

---

#### Pasul 3: Derivarea legaturii dintre $P$ si functiile $\phi$

1. Pornim de la forma propusa:
   $$
   P(A = x_A, B = x_B, C = x_C) = \frac{1}{k} \phi_{A,B}(x_A, x_B) \phi_{B,C}(x_B, x_C).
   $$

2. Calculam probabilitatea marginala $P(B = x_B)$:
   $$
   P(B = x_B) = \sum_{x_A, x_C} P(A = x_A, B = x_B, C = x_C).
   $$
   Substituind:
   $$
   P(B = x_B) = \frac{1}{k} \sum_{x_A, x_C} \phi_{A,B}(x_A, x_B) \phi_{B,C}(x_B, x_C).
   $$

3. Verificam independenta conditionata $A \perp C \mid B$:
   $$
   P(A = x_A, C = x_C \mid B = x_B) = \frac{P(A = x_A, B = x_B, C = x_C)}{P(B = x_B)}.
   $$
   Substituind expresiile:
   $$
   P(A = x_A, C = x_C \mid B = x_B) = \frac{\phi_{A,B}(x_A, x_B) \phi_{B,C}(x_B, x_C)}{\sum_{x_A, x_C} \phi_{A,B}(x_A, x_B) \phi_{B,C}(x_B, x_C)}.
   $$

4. Factorizam termenii:
   $$
   P(A = x_A \mid B = x_B) = \frac{\phi_{A,B}(x_A, x_B)}{\sum_{x_A} \phi_{A,B}(x_A, x_B)},
   $$
   $$
   P(C = x_C \mid B = x_B) = \frac{\phi_{B,C}(x_B, x_C)}{\sum_{x_C} \phi_{B,C}(x_B, x_C)}.
   $$

Aceasta factorizare arata ca $P(A = x_A, C = x_C \mid B = x_B)$ poate fi exprimata ca produsul probabilitatilor marginale conditionate, ceea ce demonstreaza ca $A \perp C \mid B$.

#2.


In [ ]:
!pip install pgmpy

In [ ]:
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.inference import BeliefPropagation

def retea_markoviana(nr_statii, muchii, prob_initiale):
    model = MarkovNetwork()

    for i in range(nr_statii):
        model.add_node(f'S{i+1}')
    model.add_edges_from(muchii)

    potential_matrice = [[0.7, 0.2, 0.1],
                         [0.2, 0.6, 0.2],
                         [0.1, 0.3, 0.6]]

    for m in muchii:
        factor = DiscreteFactor(
            variables=m,
            cardinality=[3, 3],
            values=potential_matrice
        )
        model.add_factors(factor)

    return model

nr_statii = 4
muchii = [('S1', 'S2'), ('S2', 'S3'), ('S3', 'S4')]
prob_initiale = {
    'S1': [0.5, 0.3, 0.2],
    'S2': [0.4, 0.4, 0.2],
    'S3': [0.6, 0.2, 0.2],
    'S4': [0.3, 0.3, 0.4]
}

retea = retea_markoviana(nr_statii, muchii, prob_initiale)

inf = BeliefPropagation(retea)
rez_map = inf.map_query()

print(rez_map)

#3.


In [ ]:
#a
import numpy as np
import matplotlib.pyplot as plt

img_originala = np.random.randint(0, 256, (5, 5))

plt.imshow(img_originala, cmap='gray')
plt.title('Imagine Originala')
plt.show()

In [ ]:
# b add_noise
def add_noise(imagine, procent_zgomot=0.1):

    img_noise = imagine.copy()
    nr_pixeli = imagine.size
    nr_pixeli_zgomotosi = int(procent_zgomot * nr_pixeli)

    pixeli_alesi = np.random.choice(nr_pixeli, nr_pixeli_zgomotosi, replace=False)

    for pixel in pixeli_alesi:
        i, j = np.unravel_index(pixel, imagine.shape)
        img_noise[i, j] = np.random.randint(0, 256)

    return img_noise

img_noise = add_noise(imagine_originala, procent_zgomot=0.1)
plt.imshow(img_noise, cmap='gray')
plt.title('Imagine Zgomotoasa')
plt.show()

In [ ]:
# c
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor

mrf = MarkovNetwork()

pixeli = ['P{}'.format(i) for i in range(img_originala.size)]
mrf.add_nodes_from(pixeli)

linii, coloane = img_originala.shape
for i in range(linii):
    for j in range(coloane):
        nod_curent = 'P{}'.format(i * coloane + j)

        if i > 0:
            nod_vecin_nord = 'P{}'.format((i - 1) * coloane + j)
            mrf.add_edge(nod_curent, nod_vecin_nord)

        if i < linii - 1:
            nod_vecin_sud = 'P{}'.format((i + 1) * coloane + j)
            mrf.add_edge(nod_curent, nod_vecin_sud)

        if j > 0:
            nod_vecin_vest = 'P{}'.format(i * coloane + j - 1)
            mrf.add_edge(nod_curent, nod_vecin_vest)

        if j < coloane - 1:
            nod_vecin_est = 'P{}'.format(i * coloane + j + 1)
            mrf.add_edge(nod_curent, nod_vecin_est)

lambda_reg = 0.5
pixeli_vecini = mrf.edges()

for nod1, nod2 in pixeli_vecini:
    factor = DiscreteFactor([nod1, nod2], [256, 256], np.exp(-lambda_reg * np.square(np.subtract.outer(np.arange(256), np.arange(256)))))
    mrf.add_factors(factor)

In [ ]:
#d
import numpy as np
from pgmpy.models import MarkovNetwork
from pgmpy.factors.discrete import DiscreteFactor

mrf = MarkovNetwork()

linii, coloane = 5, 5

pixeli = ['P{}'.format(i * coloane + j) for i in range(linii) for j in range(coloane)]
mrf.add_nodes_from(pixeli)

for i in range(linii):
    for j in range(coloane):
        nod_curent = 'P{}'.format(i * coloane + j)

        if i > 0:
            nod_vecin_nord = 'P{}'.format((i - 1) * coloane + j)
            mrf.add_edge(nod_curent, nod_vecin_nord)

        if i < linii - 1:
            nod_vecin_sud = 'P{}'.format((i + 1) * coloane + j)
            mrf.add_edge(nod_curent, nod_vecin_sud)

        if j > 0:
            nod_vecin_vest = 'P{}'.format(i * coloane + j - 1)
            mrf.add_edge(nod_curent, nod_vecin_vest)

        if j < coloane - 1:
            nod_vecin_est = 'P{}'.format(i * coloane + j + 1)
            mrf.add_edge(nod_curent, nod_vecin_est)

lambda_reg = 0.5
for nod1, nod2 in mrf.edges():
    factor = DiscreteFactor([nod1, nod2], [256, 256], np.exp(-lambda_reg * np.square(np.subtract.outer(np.arange(256), np.arange(256)))))
    mrf.add_factors(factor)

from pgmpy.inference import BeliefPropagation

infer = BeliefPropagation(mrf)
map_estimates = infer.map_query(variables=pixeli)

imagine_estimata = np.zeros_like(imagine_zgomot)
for i in range(nr_linii):
    for j in range(nr_coloane):
        pixel_id = 'P{}'.format(i * nr_coloane + j)
        imagine_estimata[i, j] = map_estimates[pixel_id]

plt.imshow(imagine_estimata, cmap='gray')
plt.title('Imagine Estimata (Denoised)')
plt.show()